In [ ]:
# Import required packages
import os
import sys
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#os.getcwd()

Mounted at /content/drive


In [ ]:
path_to_utils='/content/drive/My Drive/NLP'
sys.path.append(path_to_utils)

In [ ]:
os.chdir(path_to_utils)

print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1qJ8h8Q6C6t_FWn4R8pQeH813twKX9HgP/NLP


In [ ]:
import pandas as pd

obt = pd.read_csv('obtained_train.csv')
prv = pd.read_csv('provided_train.csv')


In [ ]:
prv['label'].value_counts()

neutral     10659
negative     7181
positive     5237
Name: label, dtype: int64

In [ ]:
obt['label'].value_counts()

neutral     12149
negative     8258
positive     6248
Name: label, dtype: int64

In [ ]:
len(obt), len(prv)

(26655, 23077)

In [ ]:
#sick sick sick no overlap in the 2 sets; makes things mildly easier since there is no worry of reply being counted twice

all_set=pd.concat([set1,set2], ignore_index=True) #must reindex for summarizer to work
all_set['id'].nunique()

34521

In [ ]:
class EnableBackwardIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.history = [None, ]
        self.i = 0

    def next(self):
        self.i += 1
        if self.i < len(self.history):
            return self.history[self.i]
        else:
            elem = next(self.iterator)
            self.history.append(elem)
            return elem

    def prev(self):
        self.i -= 1
        if self.i == 0:
            raise StopIteration
        else:
            return self.history[self.i]

In [ ]:
def summarizer(all_set):
  train_agglabeled = pd.DataFrame(columns=['label', 'id'])
  it = iter(range(0, len(all_set)))
  epsilon=1e-15

  for tweet in tqdm(it):

          #tweet: index
          #print(tweet)
          label_pos = 0   # total number of positive labels for each tweet
          label_neg = 0
          label_neut = 0

          label = all_set.iloc[tweet][0]
          tweet_id = all_set.iloc[tweet][1]

          if label == 'positive':
              label_pos += 1
          elif label == 'negative':
              label_neg += 1
          elif label == 'neutral':
              label_neut += 1

          while True:
              tweet = tweet + 1 #next(it)
              #print(tweet)
              #print(tweet_id)
              if tweet_id == all_set.iloc[tweet][1]:

                  tweet=next(it)
                  #print(tweet)
                  if all_set.iloc[tweet][0] == 'positive':
                      label_pos += 1
                  elif all_set.iloc[tweet][0] == 'negative':
                      label_neg += 1
                  elif all_set.iloc[tweet][0] == 'neutral':
                      label_neut += 1
              else:
                  break
              if tweet + 1 >= len(all_set):
                  break
          var = {label_pos: "positive", label_neg: "negative", label_neut: "neutral"}

          # our proposed algorithm

          #print(label_pos,label_neg,label_neut)
          overall = label_pos + label_neg + label_neut
          #print(overall)
          if (label_neut/(overall + epsilon)) > 0.85:
              label_final = var.get(label_neut)
          else:
              if (label_pos/(label_neg + epsilon)) > 1.5:
                  label_final = var.get(label_pos)
              elif (label_neg/(label_pos + epsilon)) > 1.6:
                  label_final = var.get(label_neg)
              else:
                  label_final = var.get(label_neut)

          #print(label_final)
          #tweet=tweet-1

          df = pd.DataFrame([[label_final, all_set['id'][tweet-1]]], columns=['label', 'id'])
          #print(df)
          train_agglabeled = train_agglabeled.append(df)
          #print(train_agglabeled)
  return train_agglabeled

In [ ]:
all_set

,label,id
0,positive,1223759356631625729
1,negative,1223752255846912000
2,neutral,1223752255846912000
3,negative,1223752255846912000
4,negative,1223752255846912000
...,...,...
1519499,neutral,1129423855989198850
1519500,neutral,1129423855989198850
1519501,neutral,1129423855989198850
1519502,positive,1129423855989198850


In [ ]:
#debugging summarizer by creating small set
small_set=all_set.iloc[[0,1,2, 1003, 1519501,1519502,1519503]]
small_set.reset_index(inplace=True, drop=True)
small_set

,label,id
0,positive,1223759356631625729
1,negative,1223752255846912000
2,neutral,1223752255846912000
3,negative,1223650405671997441
4,neutral,1129423855989198850
5,positive,1129423855989198850
6,positive,1129423855989198850


In [ ]:
df=summarizer(small_set)
df

4it [00:00, 345.94it/s]


,label,id
0,positive,1223759356631625729
0,neutral,1223752255846912000
0,negative,1223650405671997441
0,positive,1129423855989198850


In [ ]:
#summarizing without sorting produces the final_train_labeled.txt data provided in kaggle with duplicate tweet ids
#train_agglabeled=summarizer(all_set)

#instead, sort tweet ids in ascending order, ignoring index and then run through summarizer
all_set_ordered=all_set.sort_values(by=['id'],ignore_index=True)
train_agg_ordered=summarizer(all_set_ordered)

#see that there are no duplicate ids
len(train_agg_ordered), train_agg_ordered['id'].nunique()

34521it [08:23, 68.57it/s] 


(34521, 34521)

In [ ]:
#still unclear where ~4000 tweets disappear in this aggregation
all_set_ordered['id'].nunique(), train_agg_ordered['id'].nunique()

(34521, 34521)

In [ ]:
#pull in txt data
reply1_tweets = pd.read_csv('train_reply1_tweets.csv')
tweets_text1=reply1_tweets[['id','text']]

reply2_tweets = pd.read_csv('train_reply2_tweets.csv')
tweets_text2=reply2_tweets[['id','text']]

tweets_text=pd.concat([tweets_text1,tweets_text2])
tweets_text

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (23,27,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,text
0,1223752255846912000,The Fox Corporation (The owners of Fox News) a...
1,1223748267609030659,The news is finally out !!!! I will be on @lov...
2,1223739174160928773,Good news! The person under investigation for ...
3,1223737953291128837,"Two avid golfers promised that, whoever died f..."
4,1223736784040923136,NEWS: Andrew Yang’s Presidential campaign rais...
...,...,...
11228,1174882255736594432,Biden leaves out that his son’s corrupt compan...
11229,1119119319823110144,So another Congperson bites the dust: this tim...
11230,1148462444026744832,The UK Ambassador to the USA used undiplomatic...
11231,1161742940319993856,What is the point of extra-time? Not sure what...


In [ ]:
#join text with ids
train_agg_ordered_txt=pd.merge(train_agg_ordered,tweets_text, on='id', how='left')
#how many ids are missing text? --is this because the tweets have since been deleted?
train_agg_ordered_txt[train_agg_ordered_txt['text'].isnull()]

,label,id,text
2,neutral,1082049696904941568,NaN
3,neutral,1083563262724186112,NaN
5,positive,1084292748105048065,NaN
9,neutral,1087477951463976960,NaN
19,negative,1088859023670095872,NaN
...,...,...,...
34502,positive,1233255791441981440,NaN
34506,neutral,1233287583075098624,NaN
34507,negative,1233312080293515265,NaN
34508,neutral,1233325826982207488,NaN


In [ ]:
#drop null values and reindex
#what is the length -- ie how many tweets do we have in this training set?
train_agg_ordered_txt=train_agg_ordered_txt.dropna()
len(train_agg_ordered_txt)

26655

In [ ]:
#lets compare this obtained training set to the one provided
train_prelabeled = pd.read_csv('train_final_label.txt', delimiter = "\t")
len(train_prelabeled), train_prelabeled['id'].nunique()


(34953, 29501)

"Training autamtically-labeled dataset: 34,953 unique tweets and ONE final automatic label (chosen based on the algorithm 1 of our paper) for every tweet. Including,
./RETWEET_data/train_final_label.txt"

Directly from kaggles data_README.txt ... this is NOT true as there are not 34953 unique tweets in the train_final_label.txt file. 29501 unique tweet ids.


In [ ]:
#Do these duplicate values for tweet id have different label column values?
dupes=train_prelabeled[train_prelabeled.duplicated('id')]
dup_first=train_prelabeled.drop_duplicates(subset=['id'],keep='first')
dup_last=train_prelabeled.drop_duplicates(subset=['id'],keep='last')

dup_compare=pd.merge(dupes['id'],dup_first, on='id', how='left')
dup_compare=pd.merge(dup_compare,dup_last, on='id', how='left',suffixes=('_first', '_last'))

In [ ]:
#there are at least 108 tweets in which there are labels that are not equal to eachother -- this does not factor in ids with more than 1 dup
dup_compare.label_first.compare(dup_compare.label_last)

,self,other
61,negative,positive
90,negative,neutral
112,negative,neutral
130,positive,negative
140,negative,neutral
...,...,...
5192,neutral,negative
5233,negative,neutral
5390,negative,neutral
5392,negative,positive


In [ ]:
#Lets add text column to provided train data and drop duplicates choosing the first label to appear for each tweet id, drop rows with NaN text as well
train_tweets = pd.read_csv('train_tweets.csv')
train_tweets=train_tweets[['id','text']]

train_prelabeled_txt=pd.merge(train_prelabeled,train_tweets, on='id', how='left')
train_w_text=train_prelabeled_txt.dropna()
train_w_text.drop_duplicates(subset=['id'],keep='first', inplace=True)

#how many tweets constitute the training set using the provided training tweet ids and labels?
len(train_w_text)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (23,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


23077

In [ ]:
#recall # of tweets that constitute obtained training set
len(train_agg_ordered_txt)

26655

Obtained training set consists of 26,655 (prev 24,142) training examples while provided training set consists of 23,007 --over 1k less (both sets deduped (where apploicable) and removed NaN text field).

In [ ]:
#lets compare the labels of obtained training set to the one provided
obtained=train_agg_ordered_txt[['label','id']]
provided=train_w_text[['label','id','text']]
compare_labels=pd.merge(obtained,provided,on='id', how='outer',suffixes=('_obt', '_prv'))

compare_labels[compare_labels['label_obt']!=compare_labels['label_prv']]

,label_obt,id,label_prv,text
1,neutral,1081465965601271808.0,negative,"As elected CEO of KDSG, I take ultimate respon..."
4,positive,1087036824042856448.0,neutral,.@AOC is only speaking facts. This is so far b...
5,neutral,1087472772987576320.0,negative,The time has come to tell the true side of his...
6,neutral,1087483752534757376.0,negative,I’m so excited that @KamalaHarris has decided ...
8,negative,1088245376828592128.0,neutral,Proof it’s all about emotion to liberals! I’m ...
...,...,...,...,...
26650,neutral,1233463115611824128.0,negative,"50 years ago today, NYC Transit PD Patrolman M..."
26651,negative,1233485144247459840.0,positive,For decades after the link between smoking and...
26652,negative,1233512637276807168.0,neutral,“Smoking cigarettes kills more Americans than ...
26654,neutral,1233533352352845824.0,positive,Vegas...... great weed smoking weather


In [ ]:
compare_labels[compare_labels['label_obt'].isnull()]

,label_obt,id,label_prv,text
26655,NaN,1229770374251978752.0,positive,Mama told me this gem from a friend’s visit. D...


In [ ]:
#didnt exist in set1 and set2 .txt files -- gonna ignore it
all_set[all_set['id']==1229770374251978752]

,label,id


In [ ]:
#prv is missing tweet ids included in the obtained
#a deeper dive in the algorithm used must reveal some kind of mechanism that drops tweet ids
compare_labels[compare_labels['label_prv'].isnull()]

,label_obt,id,label_prv,text
4122,positive,1140380226263932928.0,NaN,NaN
10143,neutral,1200727082432516096.0,NaN,NaN
11086,neutral,1210274341423898624.0,NaN,NaN
11241,positive,1217890767408070656.0,NaN,NaN
11244,neutral,1219364538631344128.0,NaN,NaN
...,...,...,...,...
26609,neutral,1233190075418021888.0,NaN,NaN
26618,negative,1233215926486491136.0,NaN,NaN
26622,negative,1233220471870521344.0,NaN,NaN
26635,positive,1233247253516759040.0,NaN,NaN


In [ ]:
just_prv_tweets=compare_labels[compare_labels['label_obt'].isnull()]
just_prv_tweets=just_prv_tweets[['label_prv', 'id', 'text']]
#just_prv_tweets=pd.merge(just_prv_tweets,train_tweets, on='id', how='left')
just_prv_tweets.rename(columns={"label_prv": "label"}, inplace=True)
just_prv_tweets

,label,id,text
26655,positive,1229770374251978752.0,Mama told me this gem from a friend’s visit. D...


In [ ]:
#save all 3 data frames: obtained_train, provided_train, supp_obt_train
#make sure to index false otherwise tabulardata function will read index column into dataloader
train_agg_ordered_txt.to_csv('obtained_train.csv', index=False)
train_w_text.to_csv('provided_train.csv', index=False)

Save test file with text (removing text NaN values)

In [ ]:
import pandas as pd

#lets look at gold test data
test = pd.read_csv('test_gold.txt', delimiter = "\t")
print(test)
print(test['id'].nunique())

         label                   id
0     negative  1222226229446750214
1     negative  1236289649737371648
2     positive  1237489705090002944
3     negative  1222258314857172993
4     negative  1236432648374857728
...        ...                  ...
1514   neutral  1222250347193872384
1515   neutral  1221756922031955968
1516  negative  1223223056279179264
1517  positive  1223783189891747843
1518   neutral  1223461056804077568

[1519 rows x 2 columns]
1519


In [ ]:
test_tweets = pd.read_csv('test_tweets.csv')
test_tweets=test_tweets[['id','text']]

test_w_text=pd.merge(test,test_tweets, on='id', how='left')

test_w_text

,label,id,text
0,negative,1222226229446750214,Deputy White House counsel Pat Philbin on the ...
1,negative,1236289649737371648,Panic buying and stockpiling of toilet roll co...
2,positive,1237489705090002944,NaN
3,negative,1222258314857172993,"It’s Super Bowl week, and Trump is using a pre..."
4,negative,1236432648374857728,Turns out Donald Trump was potentially exposed...
...,...,...,...
1514,neutral,1222250347193872384,NaN
1515,neutral,1221756922031955968,MORE: Men divide 56-39% in favor of Pres. Trum...
1516,negative,1223223056279179264,NaN
1517,positive,1223783189891747843,Thank you everyone for the nice birthday wishe...


In [ ]:
#how many NaN values?
test_w_text[test_w_text['text'].isnull()]

,label,id,text
2,positive,1237489705090002944,NaN
19,negative,1223428850610245633,NaN
21,positive,1234588287269920769,NaN
26,positive,1238384745261793280,NaN
30,neutral,1234567117493669889,NaN
...,...,...,...
1502,negative,1236594724951265280,NaN
1503,positive,1224223104060329986,NaN
1514,neutral,1222250347193872384,NaN
1516,negative,1223223056279179264,NaN


In [ ]:
#dropnull values from dataframe and save
test_w_text.dropna(inplace=True)
test_w_text.reset_index(drop=True, inplace=True)
test_w_text.to_csv('test_w_text.csv', index=False)
test_w_text

,label,id,text
0,negative,1222226229446750214,Deputy White House counsel Pat Philbin on the ...
1,negative,1236289649737371648,Panic buying and stockpiling of toilet roll co...
2,negative,1222258314857172993,"It’s Super Bowl week, and Trump is using a pre..."
3,negative,1236432648374857728,Turns out Donald Trump was potentially exposed...
4,positive,1233925225697488901,Had a totally surreal author moment at B&amp;N...
...,...,...,...
1276,positive,1236253264615804928,Mexican primetime television levelled up in 20...
1277,positive,1234218004923043841,Those moments of season and spirit when even t...
1278,positive,1224379446997798912,Happy Birthday to me....\nCommitted! #GatorGan...
1279,neutral,1221756922031955968,MORE: Men divide 56-39% in favor of Pres. Trum...
